In [68]:
READ = False

In [69]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import PIL.ImageOps
from IPython.display import display 
import random
import json
from os import listdir
from os.path import isfile, join

In [70]:
def get_trait(attribute):
    mypath= f"./trait-layers/{attribute}"
    return [f[:-4] for f in listdir(mypath) if isfile(join(mypath, f))]

def list_prob(length):
    list_prob = [100//length for i in range(length - 1)]
    list_prob.append(100 - list_prob[0]*len(list_prob))
    return list_prob


In [71]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%
traits = ["Background", "Body", "Earrings", "Eyes", "Face", "Head", "Outfit"]

Background = get_trait("Background")
Background_weights = list_prob(len(Background))
Background_files = {}

Body = get_trait("Body")
Body_weights = list_prob(len(Body))
Body_files = {}

Earrings = get_trait("Earrings")
Earrings_weights = list_prob(len(Earrings))
Earrings_files = {}

Eyes = get_trait("Eyes")
Eyes_weights = list_prob(len(Eyes))
Eyes_files = {}

Face = get_trait("Face")
Face_weights = list_prob(len(Face))
Face_files = {}

Head = get_trait("Head")
Head_weights = list_prob(len(Head))
Head_files = {}

Outfit = get_trait("Outfit")
Outfit_weights = list_prob(len(Outfit))
Outfit_files = {}

# Dictionary variable for each trait. 
# Eech trait corresponds to its file name
for cat in traits:
    for cat_trait in globals()[cat]:
        globals()[cat+"_files"][cat_trait] = cat_trait

In [72]:
## Generate Traits
TOTAL_IMAGES = 20 # Number of random unique images we want to generate

all_images = [] 
# A recursive function to generate unique image combinations
def create_new_image():
    new_image = {}
    # For each trait category, select a random trait based on the weightings 
    for trait in traits:
        new_image [trait] = random.choices(globals()[trait], globals()[trait+'_weights'])[0]
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    new_trait_image = create_new_image()
    all_images.append(new_trait_image)


In [73]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [74]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [75]:
# print(all_images)

In [76]:
#### Generate Metadata for all Traits 
if not READ:
    METADATA_FILE_NAME = './metadata_new/all-traits.json'; 
    with open(METADATA_FILE_NAME, 'w') as outfile:
        json.dump(all_images, outfile, indent=4)

In [77]:
#### Generate Images
import textwrap
f = open('./metadata_new/all-traits.json',) 
all_images = json.load(f)
for item in all_images:
    bg = Image.open(f"./trait-layers/Background/{Background_files[item['Background']]}.png").convert('RGBA')

    layers = []

    layers.append(Image.open(f"./trait-layers/Body/{Body_files[item['Body']]}.png").convert('RGBA'))
    layers.append(Image.open(f"./trait-layers/Outfit/{Outfit_files[item['Outfit']]}.png").convert('RGBA'))
    layers.append(Image.open(f"./trait-layers/Head/{Head_files[item['Head']]}.png").convert('RGBA'))
    layers.append(Image.open(f"./trait-layers/Head/{Head_files[item['Head']]}.png").convert('RGBA'))
    layers.append(Image.open(f"./trait-layers/Face/{Face_files[item['Face']]}.png").convert('RGBA'))
    layers.append(Image.open(f"./trait-layers/Earrings/{Earrings_files[item['Earrings']]}.png").convert('RGBA'))

    #Create each composite
    render = bg.copy()
    for layer in layers:
        render.paste(layer, (0, 0), layer)

    render = render.convert('RGB')

    #Convert to RGB
    file_name = str(item["tokenId"]) + ".png"
    render.save("./renders/" + file_name)

In [78]:
#### Generate Metadata for each Image    

f = open('./metadata_new/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ipfs://set_base_URI/"
PROJECT_NAME = "NFT #"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "description": "Add a description here.",
        "attributes": []
    }

    for trait in traits:
        token["attributes"].append(getAttribute(trait, i[trait]))

    with open('./metadata_new/' + str(token_id) + '.json', 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()

In [79]:
cols = 3
rows = 3
im_size = Image.open("./renders/0.png").convert("RGBA")
w, h = im_size.size
im_size.close()

grid = Image.new('RGB', size=(cols*w, rows*h))
for i in range(cols*rows):
    img_i = Image.open(f"./renders/{i}.png").convert("RGBA")
    grid.paste(img_i, box=(i%cols*w, i//cols*h))
    img_i.close()

grid.save("./grid.png","PNG")
grid.close()